In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.embeddings import OpenAIEmbeddings
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
loader = PyPDFLoader("Python Basics.pdf")
pages = loader.load()

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(pages)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embedding_model = OpenAIEmbeddings()

In [10]:
vectorstore = Chroma.from_documents(documents=chunks,embedding=embedding_model,persist_directory="./chroma_db")
vectorstore.persist()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# Reconnect to persisted vectorstore
vectordb = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_model
)

retriever = vectordb.as_retriever(search_type="similarity", k=4)

# Define system + context-aware prompt
template = """You are a helpful legal assistant. Use the context from the retrieved documents to answer the user's question.
If you don't know the answer, just say so. Do not make anything up.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


C:\Users\sjogu\AppData\Local\Temp\ipykernel_29952\3927232654.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [13]:
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [14]:
while True:
    user_query = input("Ask your question (or type 'exit'): ")
    if user_query.lower() == "exit":
        break
    answer = rag_chain.run(user_query)
    print(f"\nAnswer:\n{answer}\n")


C:\Users\sjogu\AppData\Local\Temp\ipykernel_29952\2340421476.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = rag_chain.run(user_query)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}